In [ ]:
# default_exp core

# deltaframe

> Builds the delta between two or more pandas dataframes

In [ ]:
#hide
from nbdev.showdoc import *

ModuleNotFoundError: No module named 'nbdev'

In [ ]:
#export
import pandas as pd
import numpy as np

In [ ]:
#export
def get_added_entries(df_old, df_new, unique_id, trans_col="transaction", trans_vaL="added"):
    """get entries in df2 which are not in df1"""
    cols = list(df_old.columns)
    new_entries = (pd
                .merge(df_old, df_new, how="outer", on=unique_id, indicator=True, suffixes=("_foo",""))
                .query('_merge == "right_only"')
                )
    new_entries = new_entries[cols] 
    new_entries[trans_col] = trans_vaL
    return new_entries

In [ ]:
df1=pd.DataFrame({
    'Date':['2013-11-24','2013-11-24','2013-11-24','2013-11-24'],
    'Fruit':['Banana','Orange','Apple','Celery'],
    'Num':[22.1,8.6,7.6,10.2],
    'Color':['Yellow','Orange','Green','Green'],
})
df2=pd.DataFrame({
    'Date':['2013-11-24','2013-11-24','2013-11-24','2013-11-25','2013-11-25', '2014-11-03'],
    'Fruit':['Orange','Apple','Celery','Apple','Orange', 'Strawberry'],
    'Num':[8.6,7.6,10.2,22.1,8.6, 4.2],
    'Color':['Orange','Green','Green','Red','Orange', 'Red'],
})

In [ ]:
added_entries = get_added_entries(df_old=df1, df_new=df2, unique_id="Fruit")
added_entries

,Date,Fruit,Num,Color,transaction
6,2014-11-03,Strawberry,4.2,Red,added


In [ ]:
#export
def get_removed_entries(df_old, df_new, unique_id, trans_col="transaction", trans_val="removed"):
    """Returns the removed entries that are not any longer in df_new"""
    cols = list(df_old.columns)
    removed_entries = (pd
                .merge(df_new, df_old, how="outer", on=unique_id, indicator=True, suffixes=("_foo",""))
                .query('_merge == "right_only"')
                )
    removed_entries = removed_entries[cols]
    removed_entries[trans_col] = trans_val
    return removed_entries

In [ ]:
removed_entries = get_removed_entries(df_old=df1, df_new=df2, unique_id="Fruit")
removed_entries

,Date,Fruit,Num,Color,transaction
6,2013-11-24,Banana,22.1,Yellow,removed


In [ ]:
#export 
def get_modified_entries(df_old, df_new, unique_id, added_entries=None, trans_col="transaction", trans_val="modified"):
    """Returns the modified entries"""
    if added_entries is not None: 
        df_new = df_new[~df_new.isin(list(added_entries[unique_id].values))].dropna()
    modified_entries = df_new[~df_new.isin(df_old)].dropna()
    modified_entries[trans_col] = trans_val
    return modified_entries

In [ ]:
modified_entries = get_modified_entries(df_old=df1, df_new=df2, unique_id="Fruit")
modified_entries

,Date,Fruit,Num,Color,transaction
3,2013-11-25,Apple,22.1,Red,modified
4,2013-11-25,Orange,8.6,Orange,modified
5,2014-11-03,Strawberry,4.2,Red,modified


In [ ]:
modified_entries = get_modified_entries(df_old=df1, df_new=df2, unique_id="Fruit", added_entries=added_entries)
modified_entries

,Date,Fruit,Num,Color,transaction
3,2013-11-25,Apple,22.1,Red,modified
4,2013-11-25,Orange,8.6,Orange,modified


In [ ]:
df1.sort_values

In [ ]:
#export 
def logging(df_log, df_old, df_new, unique_id, trans_col="transaction", trans_val_added="added", trans_val_removed="removed", trans_val_modified="modified", sort_by=None):
    if df_log is None:
        df_log = df_old.copy()
        df_log[trans_col] = trans_val_added
    else:
        subset = list(df_log.columns)
        subset.remove(trans_col)
        added_entries = get_added_entries(df_old=df_old, df_new=df_new, unique_id=unique_id, trans_col=trans_col, trans_vaL=trans_val_added)
        removed_entries = get_removed_entries(df_old=df_old, df_new=df_new, unique_id=unique_id, trans_col=trans_col, trans_val=trans_val_removed)
        modified_entries = get_modified_entries(df_new=df_new, df_old=df_old, unique_id=unique_id, added_entries=added_entries, trans_col=trans_col, trans_val=trans_val_modified)
        df_log = df_log.append(modified_entries, ignore_index=True)
        df_log = df_log.drop_duplicates(subset=subset, keep="first")
        df_log = df_log.append(added_entries, ignore_index=True)
        df_log = df_log.append(removed_entries, ignore_index=True)  
    if sort_by:
        df_log = df_log.sort_values(by=sort_by)
    return df_log 

In [ ]:
df_log = logging(df_log=None, df_old=df1, df_new=df2, unique_id="Fruit")
df_log

,Date,Fruit,Num,Color,transaction
0,2013-11-24,Banana,22.1,Yellow,added
1,2013-11-24,Orange,8.6,Orange,added
2,2013-11-24,Apple,7.6,Green,added
3,2013-11-24,Celery,10.2,Green,added


In [ ]:
df_log = logging(df_log=df_log, df_old=df1, df_new=df2, unique_id="Fruit")
df_log

,Date,Fruit,Num,Color,transaction
0,2013-11-24,Banana,22.1,Yellow,added
1,2013-11-24,Orange,8.6,Orange,added
2,2013-11-24,Apple,7.6,Green,added
3,2013-11-24,Celery,10.2,Green,added
4,2013-11-25,Apple,22.1,Red,modified
5,2013-11-25,Orange,8.6,Orange,modified
6,2014-11-03,Strawberry,4.2,Red,added
7,2013-11-24,Banana,22.1,Yellow,removed


In [ ]:
df_log = logging(df_log=df_log, df_old=df1, df_new=df2, unique_id="Fruit", sort_by=["Date"])
df_log

,Date,Fruit,Num,Color,transaction
0,2013-11-24,Banana,22.1,Yellow,added
1,2013-11-24,Orange,8.6,Orange,added
2,2013-11-24,Apple,7.6,Green,added
3,2013-11-24,Celery,10.2,Green,added
8,2013-11-24,Banana,22.1,Yellow,removed
4,2013-11-25,Apple,22.1,Red,modified
5,2013-11-25,Orange,8.6,Orange,modified
6,2014-11-03,Strawberry,4.2,Red,added
7,2014-11-03,Strawberry,4.2,Red,added
